In [1]:
%load_ext autoreload
%autoreload 1

import sys
sys.path.append("../utils/")

from sklearn.preprocessing import StandardScaler, Normalizer, MinMaxScaler, MaxAbsScaler, RobustScaler

from GraphAPI import GraphCreator
from graph_helpers import *
from evaluations import *

%aimport GraphAPI
%aimport graph_helpers
%aimport evaluations

## Generating Graph from Entry Point

1. We initialize our GraphCreator class and check how many new nodes we will need to query. 

In [2]:
# include_see_also=False used for validation below.
# in deployment, include_see_also should be set to True
gc = GraphCreator("Function (music)",
                  include_see_also=False, max_recursive_requests=50)
print("Number of Links to Search:", len(gc.next_links), "\n\n")
print(list(gc.primary_nodes.keys()), "\n\n")
print(gc.see_also_articles)

Number of Links to Search: 178 


['Chord (music)', 'Scale degree', 'Tonal centre'] 


['Common practice period', 'Constant structure', 'Diatonic and chromatic', 'Nondominant seventh chord', 'Secondary dominant', 'Subsidiary chord', 'Roman numeral analysis']


2. We query all the nodes linked to/from the entry point (expand our network one level for each node).

In [3]:
gc.expand_network_threaded(threads=20, chunk_size=1)
print("Number of Links After Expansion: ", len(gc.graph.nodes))

Number of Links After Expansion:  54655


3. Since some nodes will likely have linked to articles through a redirect link, we need to traverse our graph and ensure that all redirects are assigned to the correct nodes. Once all redirects have been dealt with, we remove any old redirect nodes. 

In [4]:
gc.redraw_redirects()

4. Edges are weighted by how many categories two connected nodes have in common. Once we have all our nodes, and we have dealt with redirects, we can add edge weights for our entire graph. 

In [5]:
gc.update_edge_weights()
gc.get_edge_weights().head()

,source_node,target_node,edge_weight
0,Ostinato,Figured bass,5
1,Figured bass,Ostinato,5
2,Circle of fifths,Parallel key,4
3,Block chord,So What chord,4
4,Parallel key,Modulation (music),4


# Getting Our Feature Set

There are two options when generating the feature set:

1. we can generate a standard feature set with only the features themselves. To do this, have the `rank` parameter set to `False`.
2. We can generate a ranked feature set (set `rank` equal to `True`). For each parameter, this will rank them in order of _best_ to _worst_ (this could be ascending or descending, depending on the context of the feature).

After running `get_features_df`, the feature set will be saved in the GraphCreator instance as `feature_df`

In [6]:
features_df = gc.get_features_df(rank=False)

# Similarity Rank

Two articles are more similar the more categories they share and the closer they are to each other. 

In [9]:
gc.rank_similarity()
gc.features_df.sort_values("similarity_rank", ascending=False)

,node,degree,category_matches_with_source,in_edges,out_edges,shared_neighbors_with_entry_score,centrality,page_rank,adjusted_reciprocity,shortest_path_length_from_entry,shortest_path_length_to_entry,jaccard_similarity,primary_link,similarity_rank
72,Function (music),240,1,116,124,1.000000,8.635168e-02,0.000618,0.100069,0.0,0.0,1.000000,0,5.337330
124,Predominant chord,117,1,16,101,0.650350,1.769306e-02,0.000114,0.010007,1.0,1.0,0.100000,0,1.398751
79,Parallel and counter parallel,220,1,100,120,0.645570,1.230823e-01,0.000526,0.081875,1.0,1.0,0.714286,0,1.345617
117,Supertonic,176,1,139,37,0.524324,8.920003e-02,0.000585,0.021833,1.0,1.0,0.574074,0,1.257941
115,Subtonic,181,1,128,53,0.507853,8.843265e-02,0.000540,0.020924,1.0,1.0,0.615894,0,1.235839
51,Secondary chord,298,1,115,183,0.570732,1.058097e-01,0.000635,0.096430,1.0,1.0,0.804688,0,1.175517
86,Mediant,212,1,167,45,0.443439,8.869096e-02,0.000696,0.020924,1.0,1.0,0.481675,0,1.140821
142,Terzschritt,49,1,21,28,0.191176,2.264421e-02,0.000176,0.015465,1.0,1.0,0.151261,0,1.134546
134,Tonicization,74,1,38,36,0.238095,1.435550e-02,0.000290,0.020014,1.0,1.0,0.203125,0,1.129988
57,Submediant,276,1,153,123,0.485981,8.824209e-02,0.000606,0.080055,1.0,1.0,0.528409,0,1.124833


# Scaling Features

We can easily scale our each of our features through the `scale_features_df` method. It will default to `Standard Scaler`, but we can specify alternate scalers in the `scaler` parameter.  

In [8]:
scaled_feature_df = gc.scale_features_df(scaler=Normalizer, copy=True) # Makes a copy of the df
scaled_feature_df.sort_values("similarity_rank", ascending=False).reset_index().drop("index", axis=1)

,node,degree,category_matches_with_source,in_edges,out_edges,shared_neighbors_with_entry_score,centrality,page_rank,adjusted_reciprocity,shortest_path_length_from_entry,shortest_path_length_to_entry,jaccard_similarity,primary_link,similarity_rank
0,Riemannian theory,0.799644,0.019991,0.259884,0.539760,3.042123e-03,1.756880e-04,2.051552e-06,0.000200,0.019991,0.019991,9.751754e-04,0.0,2.191439e-02
1,Terzschritt,0.813246,0.016597,0.348534,0.464712,3.172929e-03,3.758228e-04,2.927237e-06,0.000257,0.016597,0.016597,2.510449e-03,0.0,1.882991e-02
2,Function (music),0.816197,0.003401,0.394495,0.421702,3.400820e-03,2.936665e-04,2.101452e-06,0.000340,0.000000,0.000000,3.400820e-03,0.0,1.815130e-02
3,Tonicization,0.816180,0.011029,0.419119,0.397060,2.626061e-03,1.583333e-04,3.202543e-06,0.000221,0.011029,0.011029,2.240359e-03,0.0,1.246316e-02
4,Predominant chord,0.752861,0.006435,0.102955,0.649906,4.184811e-03,1.138497e-04,7.364371e-07,0.000064,0.006435,0.006435,6.434709e-04,0.0,9.000554e-03
5,Supertonic,0.774263,0.004399,0.611492,0.162771,2.306618e-03,3.924106e-04,2.574233e-06,0.000096,0.004399,0.004399,2.525479e-03,0.0,5.533959e-03
6,Subtonic,0.794049,0.004387,0.561538,0.232512,2.227959e-03,3.879552e-04,2.367811e-06,0.000092,0.004387,0.004387,2.701935e-03,0.0,5.421641e-03
7,Parallel and counter parallel,0.815342,0.003706,0.370610,0.444732,2.392546e-03,4.561552e-04,1.948028e-06,0.000303,0.003706,0.003706,2.647214e-03,0.0,4.986992e-03
8,Mediant,0.774824,0.003655,0.610357,0.164467,1.620694e-03,3.241504e-04,2.545170e-06,0.000076,0.003655,0.003655,1.760442e-03,0.0,4.169508e-03
9,Submediant,0.814877,0.002952,0.451725,0.363151,1.434836e-03,2.605304e-04,1.789712e-06,0.000236,0.002952,0.002952,1.560102e-03,0.0,3.321015e-03


___
# Validation

Here, we _validate_ our results. For many articles, we already have some user defined links that are highly related to the present article. These are found in the **See Also** section of several Wikipedia articles (some pages do not have them). These are not ordered in importance in any meaningful way, and there are no rating scores.

The intuition in this validation is as follows: 

> _Given that we know some articles are highly related from user input, if the recommendations provided by this system are valid, we would expect to see those **See Also** links ranked relatively high on our list._ 

_Note: This validation is not meant as **confirmation** or **evaluation** of the results. It only provides us one way of telling if the results we are seeing are reasonably valid. It is important to note that we cannot compare these results across two different articles, as those would be two entirely different network structures, likely with different human labeled links._  


In [ ]:
evaluate_metrics(scaled_feature_df, 
                 on=["similarity_rank", "centrality", "adjusted_reciprocity", "page_rank", "shortest_path_length_from_entry", "jaccard_similarity"], 
                 targets=gc.see_also_articles)

The chart generated above compares different ranking metrics (left index) for a given article. The most important column, `score`, provides a fast way for us to compare these different metrics. 

For example, if we see a _score_ of 0.98 for a given ranking metric, The following statement would be true:

> All of the human labeled **See Also** links are present within the top 98% of our recommendations. 

Since the human labeled links comprise a range, it is not possible to get a score of 100%. The `max score possible` column indicates the score that would be achieved if all the human labeled _See Also_ links appeared at the top of our recommendations without any other links intervening.   

The `difference` column is an alternative way of looking at the score. If we had a 0.02 in this column, we could say:

> All the human labeled **See Also** links are contained within the top 2% of our recommendations. 

`Total targets` is the number of human labeled _See Also_ links. 

Because it is possible that different metrics could have similar scores, we want a way to break down the dispersion of the known related links to see if one metric does perform better than another. The trailing four columns provide us with a course way of measuring this dispersion. 

Each of these columns indicates the percentage of human labeled _See Also_ links captured within a given percentage of the top of our recommendations. For example, if we see a 0.92 in the `% targets in the top 1%` columns, we could say:

> 92% of the human labeled **See Also** links appear in the top 1% of our recommendations. 

The value of these columns is a follows - If two ranking metrics have similar scores, we _might_ consider the better performing one to be the one in which the majority of the human labeled links are higher in our recommendation list. 

___